Precision and recall
https://insidelearningmachines.com/precisionk_and_recallk/#:~:text=Precision%40k%20and%20Recall%40k%20are%20metrics%20used%20to%20evaluate,end%20user%20by%20the%20model.

In [1]:
# imports
import pandas as pd
import numpy as np
from typing import List
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings


In [2]:
behaviors = pd.read_csv("MIND/behaviors_testdataset.csv", sep=';', header=None)
behaviors.columns =['User', 'Time', 'ID', 'Impressions'] 
behaviors = behaviors.drop(['Time', 'ID'], axis=1)
behaviors.head()

,User,Impressions
0,U13000,N7482-1 N6379-0
1,U14000,N1145-1 N34930-1 N53526-0 N3574-0 N751-0 N2202...
2,U15000,N4303-1 N58730-0 N64130-0 N38064-0 N49210-0 N92-0
3,U13740,N55689-1 N35729-0
4,U91836,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...


In [3]:
users = pd.read_csv("data_embeddings/users_emb_test.csv") #document with user interactions
users.head()

,User,ID,Interactions_emb
0,U10045,N52865 N22570 N37481 N55189 N16158 N63276 N150...,"[-0.016629040241241455, -0.011621594429016113,..."
1,U11306,N18064 N38868 N60340 N31801 N39778 N54842 N110...,"[-0.03462968394160271, -0.024745389819145203, ..."
2,U13000,N42782 N18445 N49749,"[-0.018553022295236588, -0.008284046314656734,..."
3,U13740,N55189 N42782 N34694 N45794 N18445 N63302 N104...,"[-0.020748168230056763, -0.02500385232269764, ..."
4,U14000,N9721 N61358 N11360 N64593 N9900 N46688,"[0.016509052366018295, 0.002539730630815029, 0..."


In [4]:
news = pd.read_csv("hybrid_test.csv") 
news.head()

,ID,category,sub_category,content,collaborative_rec,content_rec,mean,two_one
0,N52865,news,newspolitics,Ohio's Reputation As a Must-Win State May No L...,0.16384,0.205185,0.184513,0.191403
1,N10270,autos,autossports,What Do Old-School Mustang Guys Think of the K...,0.16384,0.253155,0.208497,0.223383
2,N21428,autos,autosnews,Ford's Mach E Electric SUV Spied Looking...Int...,0.16384,0.267140,0.215490,0.232706
3,N1406,sports,football_nfl,NFL Films signed off on airing of Sam Darnold'...,0.16384,0.231320,0.197580,0.208827
4,N32027,sports,football_nfl,Payton declines to name Saints starting QB for...,0.16384,0.235284,0.199562,0.211469


Calculate the proportion of -1 and 0 in 'Impressions' 

In [5]:
# Function to count the number of suffixes
def count_suffixes(row, suffix):
    impressions = row['Impressions'].split()
    count = sum(1 for imp in impressions if imp.endswith(suffix))
    return count

In [6]:
# Counting "-1" and "-0" suffixes
behaviors['-1 Count'] = behaviors.apply(lambda row: count_suffixes(row, '-1'), axis=1)
behaviors['-0 Count'] = behaviors.apply(lambda row: count_suffixes(row, '-0'), axis=1)

# Total count across all users
total_minus_1 = behaviors['-1 Count'].sum()
total_minus_0 = behaviors['-0 Count'].sum()

print("Total -1 count:", total_minus_1)
print("Total -0 count:", total_minus_0)

Total -1 count: 29
Total -0 count: 1018


In [7]:
total_minus_1/total_minus_0

0.02848722986247544

Calculate precision and recall

In [8]:
def create_user_df(input_df, user):
    user_row = input_df[input_df['User'] == user]

    if user_row.empty:
        return None

    impressions = user_row['Impressions'].values[0].split()

    news_ids = []
    true_values = []

    for impression in impressions:
        news_id, true_value = impression.split('-')
        news_ids.append(news_id)
        true_values.append(int(true_value))

    user_df = pd.DataFrame({'ID': news_ids, 'true_value': true_values})
    return user_df

In [9]:
# Assuming you have a DataFrame named 'users' with a 'User' column
users_list = users['User'].unique().tolist()

# Print the resulting list
print(users_list)

['U10045', 'U11306', 'U13000', 'U13740', 'U14000', 'U15000', 'U17841', 'U19739', 'U29155', 'U34670', 'U38627', 'U46596', 'U53231', 'U63162', 'U73700', 'U79199', 'U8125', 'U8312', 'U8355', 'U89744', 'U91836', 'U92486']


In [14]:
#recommder_type = [content, collab, hybrid]
def precision_and_recall (list_name, recommender_type):
    if recommender_type == 'content':
        column = 'content_rec'
    if recommender_type == 'collab':
        column = 'collaborative_rec'
    if recommender_type == 'hybrid':
        column = 'mean'
    
    all_precision = []
    all_recall = []
    
    user_df = list_name
    
    for i in users_list:
        
        user_df = create_user_df(behaviors, i)
        news = pd.read_csv("hybrid_recommendations/" + i + "_hybr.csv") 
        
        id_to_new_column = news.set_index('ID')[column].to_dict()
        user_df[column] = user_df['ID'].map(id_to_new_column)
        
        three_percent = np.percentile(news[column], 3)
        
        user_df['predicted_value'] = user_df[column].apply(lambda x: 0 if x >= three_percent else 1)
        
        # Assuming df is your DataFrame
        true_values = user_df['true_value']
        predicted_values = user_df['predicted_value']

        # Calculate precision
        precision = precision_score(true_values, predicted_values, average='weighted') #'weighted' or 'macro'
        recall =recall_score(true_values, predicted_values, average='weighted') #'weighted' or 'macro'
        
        all_precision.append(precision)
        all_recall.append(recall)
        #print(user_df)
    print(np.mean(all_precision))
    print(np.mean(all_recall))

In [15]:
warnings.filterwarnings("ignore")

In [16]:
precision_and_recall(users_list, 'content')

0.7409592462454685
0.7879524267752245


In [17]:
precision_and_recall(users_list, 'collab')

0.7281155800806737
0.8360542291069512
